In [1]:
from abc import ABCMeta, abstractmethod
from collections import namedtuple
from itertools import chain, repeat
from pprint import pformat, pprint
from typing import (
    AbstractSet, Any, Callable, Collection, Dict, Generator, Generic, Iterable, List, Mapping,
    NamedTuple, Optional, Tuple, TypeVar, Union
)

from graphql import GraphQLList, GraphQLString, parse
from graphql.utilities.build_ast_schema import build_ast_schema
from graphql_compiler.compiler.blocks import (
    Backtrack, CoerceType, ConstructResult, EndOptional, Filter, GlobalOperationsStart, 
    MarkLocation, Recurse, QueryRoot, Traverse
)
from graphql_compiler.compiler.compiler_entities import BasicBlock, Expression
from graphql_compiler.compiler.expressions import (
    BinaryComposition, ContextField, ContextFieldExistence, Literal, LocalField, 
    OutputContextField, TernaryConditional, Variable
)
from graphql_compiler.compiler.compiler_frontend import IrAndMetadata, graphql_to_ir
from graphql_compiler.compiler.helpers import Location, get_only_element_from_collection
from graphql_compiler.interpreter.api import interpret_ir
from graphql_compiler.interpreter.typedefs import DataContext, EdgeInfo, InterpreterAdapter
from graphql_compiler.schema import GraphQLDate, GraphQLDateTime, GraphQLDecimal
from graphql_compiler.tests.test_helpers import SCHEMA_TEXT

In [2]:
vertices = {
    'Animal': [
        {'name': 'Scooby Doo', 'uuid': '1001'},
        {'name': 'Hedwig', 'uuid': '1002'},
        {'name': 'Beethoven', 'uuid': '1003'},
        {'name': 'Pongo', 'uuid': '1004'},
        {'name': 'Perdy', 'uuid': '1005'},
        {'name': 'Dipstick', 'uuid': '1006'},
        {'name': 'Dottie', 'uuid': '1007'},
        {'name': 'Domino', 'uuid': '1008'},
        {'name': 'Little Dipper', 'uuid': '1009'},
        {'name': 'Oddball', 'uuid': '1010'},
    ],
}
edges = {
    'Animal_ParentOf': [
        ('1004', '1006'),
        ('1005', '1006'),
        ('1006', '1008'),
        ('1006', '1009'),
        ('1006', '1010'),
        ('1007', '1008'),
        ('1007', '1009'),
        ('1007', '1010'),
    ],
}

vertices_by_uuid = {
    vertex['uuid']: vertex
    for vertex in chain.from_iterable(vertices.values())
}


class InMemoryAdapter(InterpreterAdapter[dict]):
    def get_tokens_of_type(
        self,
        type_name: str, 
        **hints
    ) -> Iterable[dict]:
        return vertices[type_name]

    def project_property(
        self,
        data_contexts: Iterable[DataContext],
        current_type_name: str,
        field_name: str,
        **hints
    ) -> Iterable[Tuple[DataContext, Any]]:
        for data_context in data_contexts:
            current_token = data_context.current_token
            current_value = current_token[field_name] if current_token is not None else None
            yield (data_context, current_value)

    def project_neighbors(
        self,
        data_contexts: Iterable[DataContext], 
        current_type_name: str,
        edge_info: EdgeInfo,
        **hints
    ) -> Iterable[Tuple[DataContext, Iterable[dict]]]:
        direction, edge_name = edge_info
        edge_data = edges[edge_name]
        
        for data_context in data_contexts:
            neighbor_tokens = []
            current_token = data_context.current_token
            if current_token is not None:
                uuid = current_token['uuid']
                if direction == 'out':
                    neighbor_tokens = [
                        vertices_by_uuid[destination_uuid]
                        for source_uuid, destination_uuid in edge_data
                        if source_uuid == uuid
                    ]
                elif direction == 'in':
                    neighbor_tokens = [
                        vertices_by_uuid[source_uuid]
                        for source_uuid, destination_uuid in edge_data
                        if destination_uuid == uuid
                    ]
                else:
                    raise AssertionError()
                
            yield (data_context, neighbor_tokens)

    def can_coerce_to_type(
        self,
        data_contexts: Iterable[DataContext], 
        current_type_name: str,
        coerce_to_type_name: str,
        **hints
    ) -> Iterable[Tuple[DataContext, bool]]:
        # TODO(predrag): See if a redesign can make this be a no-op again.
        return zip(data_contexts, repeat(True))

In [3]:
schema = build_ast_schema(parse(SCHEMA_TEXT))

In [4]:
def recursion_sanity_check():
    depth_2 = """
{
    Animal {
        name @output(out_name: "start")
        
        out_Animal_ParentOf @recurse(depth: 2) {
            name @output(out_name: "first_recursion")
            
            in_Animal_ParentOf @recurse(depth: 2) {
                name @output(out_name: "second_recursion")
            }
        }
    }
}"""
    depth_3 = """
{
    Animal {
        name @output(out_name: "start")
        
        out_Animal_ParentOf @recurse(depth: 3) {
            name @output(out_name: "first_recursion")
            
            in_Animal_ParentOf @recurse(depth: 3) {
                name @output(out_name: "second_recursion")
            }
        }
    }
}"""
    depth_4 = """
{
    Animal {
        name @output(out_name: "start")
        
        out_Animal_ParentOf @recurse(depth: 4) {
            name @output(out_name: "first_recursion")
            
            in_Animal_ParentOf @recurse(depth: 4) {
                name @output(out_name: "second_recursion")
            }
        }
    }
}"""
    query_arguments = {}
    ir_and_metadata = graphql_to_ir(schema, depth_2)
    expected_result = list(interpret_ir(InMemoryAdapter(), ir_and_metadata, query_arguments))
    
    for query in (depth_3, depth_4):
        ir_and_metadata = graphql_to_ir(schema, query)
        actual_result = list(interpret_ir(InMemoryAdapter(), ir_and_metadata, query_arguments))
        
        if expected_result != actual_result:
            raise AssertionError(f"Result mimatch on query: {query}")
            
    print("Success!")
            
recursion_sanity_check()

{'runtime_arg_hints': {}, 'used_property_hints': frozenset({'name'}), 'filter_hints': [], 'neighbor_hints': [(('out', 'Animal_ParentOf'), None)]}
{'runtime_arg_hints': {}, 'used_property_hints': frozenset({'name'}), 'filter_hints': [], 'neighbor_hints': [(('out', 'Animal_ParentOf'), None)]}
{'runtime_arg_hints': {}, 'used_property_hints': frozenset({'name'}), 'filter_hints': [], 'neighbor_hints': [(('out', 'Animal_ParentOf'), None)]}
Success!


In [5]:
query = '''
{
    Animal {
        name @output(out_name: "animal_name")
        uuid @output(out_name: "animal_uuid")
    }
}
'''
query_arguments = {}

In [6]:
query = '''
{
    Animal {
        name @output(out_name: "parent_name")

        out_Animal_ParentOf {
            name @output(out_name: "child_name")
        }
    }
}
'''
query_arguments = {}

In [7]:
query = '''{
    Animal {
        name @output(out_name: "parent_name")
        
        out_Animal_ParentOf @optional {
            name @filter(op_name: "in_collection", value: ["$child_names"])
                 @output(out_name: "child_name")
        }
    }
}'''
query_arguments = {
    "child_names": ['Domino', 'Dipstick', 'Oddball'],
}

In [8]:
query = '''{
    Animal {
        name @output(out_name: "ancestor_or_self_name")
        
        out_Animal_ParentOf @recurse(depth: 4) {
            name @filter(op_name: "in_collection", value: ["$names"])
                 @output(out_name: "descendant_or_self_name")
        }
    }
}'''
query_arguments = {
    "names": ['Domino', 'Dipstick', 'Oddball'],
}

In [9]:
query = '''{
    Animal {
        name @output(out_name: "ancestor_or_self_name")
        
        out_Animal_ParentOf @recurse(depth: 1) {
            name @filter(op_name: "in_collection", value: ["$names"])
                 @output(out_name: "descendant_or_self_name")
        }
    }
}'''
query_arguments = {
    "names": ['Domino', 'Dipstick', 'Oddball'],
}

In [10]:
query = '''
{
    Animal {
        name @output(out_name: "parent_name")

        out_Animal_ParentOf {
            name @filter(op_name: "in_collection", value: ["$child_names"])
                 @output(out_name: "child_name")
        }
    }
}
'''
query_arguments = {
    "child_names": ['Domino', 'Dipstick', 'Oddball'],
}

In [11]:
query = '''{
    Animal {
        name @output(out_name: "ancestor_or_self_name")
        
        out_Animal_ParentOf @recurse(depth: 4) {
            name @output(out_name: "descendant_or_self_name")
        }
    }
}'''
query_arguments = {}

In [12]:
query = '''{
    Animal {
        name @output(out_name: "start") @filter(op_name: "=", value: ["$start"])
        
        in_Animal_ParentOf @recurse(depth: 4) {
            name @output(out_name: "first_recursion")
        }
    }
}'''
query_arguments = {
    "start": "Dipstick"
}

In [13]:
query = '''{
    Animal {
        name @output(out_name: "start") @filter(op_name: "=", value: ["$start"])
        
        out_Animal_ParentOf @recurse(depth: 3) {
            name @output(out_name: "first_recursion")
            
            in_Animal_ParentOf @recurse(depth: 3) {
                name @output(out_name: "second_recursion")
            }
        }
    }
}'''
query_arguments = {
    "start": "Perdy"
}

In [14]:
query = '''{
    Animal {
        name @output(out_name: "start")
        
        out_Animal_ParentOf @recurse(depth: 3) {
            name @output(out_name: "first_recursion")
            
            in_Animal_ParentOf @recurse(depth: 3) {
                name @output(out_name: "second_recursion")
            }
        }
    }
}'''
query_arguments = {}

In [15]:
query = '''
{
    Animal {
        name @output(out_name: "grandparent_name")

        out_Animal_ParentOf {
            name @output(out_name: "parent_name")
            
            out_Animal_ParentOf {
                name @output(out_name: "child_name")
            }
        }
    }
}
'''
query_arguments = {}

In [17]:
query = '''
{
    Animal {
        name @output(out_name: "animal_name") @filter(op_name: "=", value: ["$name"])
        uuid @output(out_name: "animal_uuid")
        
        out_Animal_ParentOf {
            name @output(out_name: "child_name")
        }
        in_Animal_ParentOf {
            name @output(out_name: "parent_name")
        }
    }
}
'''
query_arguments = {
    "name": "Dipstick",
}

In [18]:
ir_and_metadata = graphql_to_ir(schema, query)
# pprint(ir_and_metadata.ir_blocks)

result = list(interpret_ir(InMemoryAdapter(), ir_and_metadata, query_arguments))
result

{'runtime_arg_hints': {'name': 'Dipstick'}, 'used_property_hints': frozenset({'uuid', 'name'}), 'filter_hints': [FilterInfo(fields=('name',), op_name='=', args=('$name',))], 'neighbor_hints': [(('out', 'Animal_ParentOf'), None), (('in', 'Animal_ParentOf'), None)]}


[{'animal_name': 'Dipstick',
  'animal_uuid': '1006',
  'child_name': 'Domino',
  'parent_name': 'Pongo'},
 {'animal_name': 'Dipstick',
  'animal_uuid': '1006',
  'child_name': 'Domino',
  'parent_name': 'Perdy'},
 {'animal_name': 'Dipstick',
  'animal_uuid': '1006',
  'child_name': 'Little Dipper',
  'parent_name': 'Pongo'},
 {'animal_name': 'Dipstick',
  'animal_uuid': '1006',
  'child_name': 'Little Dipper',
  'parent_name': 'Perdy'},
 {'animal_name': 'Dipstick',
  'animal_uuid': '1006',
  'child_name': 'Oddball',
  'parent_name': 'Pongo'},
 {'animal_name': 'Dipstick',
  'animal_uuid': '1006',
  'child_name': 'Oddball',
  'parent_name': 'Perdy'}]